<p style="text-align: center">
    <img src="../../assets/images/untref-logo-negro.svg" style="height: 50px;" />
</p>

<h3 style="text-align: center">Estructuras de Datos</h3>

<h2 style="text-align: center">Clase 4: Organización de Datos</h3>

## Organización de datos
Distintas formas de organizar la información para guardarla en un archivo y poder recuperarla. Se acostumbra hablar de la organización lógica de un archivo. 
Vamos a analizar distintas estrategias.

Supongamos que queremos almacenar información de contactos: nombre, calle y ciudad.
Una primera aproximación podría ser guardar los datos sin ninguna organización lógica, simplemente escribiendo la información en el archivo.

In [ ]:
class Agenda:
    def __init__(self, archivo):
        self.archivo = archivo

    def guardar_contacto(self, nombre, calle, ciudad):
        with open(self.archivo, "a") as datos:
            datos.write(nombre)
            datos.write(calle)
            datos.write(ciudad)

In [ ]:
agenda = Agenda("agenda_v1")
agenda.guardar_contacto("Ana Perez", "San Martin 2301", "Caseros")
agenda.guardar_contacto("Juan Pedro Ares", "Arenales 1208", "Ciudad de Buenos Aires")

In [ ]:
for linea in open("agenda_v1", "r"):
    print(linea)

>Se perdió la integridad lógica de los datos, no sabemos donde empieza y termina cada unidad de información
- **Campo:** Es la mínima unidad de información. En este ejemplo tenemos tres campos: *nombre*, *calle* y *ciudad*
- **Registro:** Conjunto de campos vinculados a la misma entidad, en este caso tenemos dos registros, el de Ana y el de Juan

### Registros de longitud fija (con campos de longitud fija)
Una forma posible sería utilizar longitudes fijas para cada campo. Por ejemplo:
- nombre: 30 posiciones
- calle: 30 posiciones
- ciudad: 20 posiciones

In [ ]:
class AgendaException(Exception):
    """Clase base para excepciones de Agenda"""

    pass

In [ ]:
import struct
import os


class Agenda:
    """Agenda con registros de tamaño fijo
    y campos de longitud fija
    """

    def __init__(self, archivo, **campos):
        """Constructor de la clase Agenda
        :param archivo: nombre del archivo donde se guardan los registros
        :param campos: diccionario con los campos de la agenda y su longitud
        """
        self._archivo = archivo
        self._campos = campos  # diccionario con los campos de la agenda y su longitud

        # "30s30s20s" -> 30 caracteres para el primer campo, 30 para el segundo y 20 para el tercero
        self._formato = "".join(f"{longitud}s" for longitud in campos.values())
        self._len_registro = struct.calcsize(self._formato)

        try:
            # obtengo el tamaño del archivo si es mayor que cero ya teniamos registros guardados
            tam_archivo = os.path.getsize(archivo)
            self._cant_registros = tam_archivo // self._len_registro
        except FileNotFoundError:
            self._cant_registros = 0

    def guardar_contacto(self, **datos):
        campos_validos = set(self._campos.keys())
        campos_recibidos = set(datos.keys())

        if campos_validos != campos_recibidos:
            raise AgendaException("Los campos recibidos no coinciden con los campos definidos en la agenda.")

        with open(self._archivo, "ab") as fd:
            valores = [campo.encode() for campo in datos.values()]
            fd.write(struct.pack(self._formato, *valores))
        self._cant_registros += 1

    def cantidad_registros(self):
        return self._cant_registros

    def campos(self):
        return self._campos

    def __iter__(self):
        return AgendaIterator(self)

In [ ]:
class AgendaIterator:
    def __init__(self, agenda):
        self._agenda = agenda
        self._index = 0

    def __next__(self):
        if self._index >= self._agenda._cant_registros:
            raise StopIteration

        with open(self._agenda._archivo, "rb") as fd:
            posicion = self._index * self._agenda._len_registro
            fd.seek(posicion)
            registro = fd.read(self._agenda._len_registro)
            self._index += 1
            if len(registro) == self._agenda._len_registro:
                valores = struct.unpack(self._agenda._formato, registro)
                resultado = tuple(valor.decode() for valor in valores)
            else:
                raise AgendaException("Error al leer el registro.")

        return resultado

In [ ]:
agenda = Agenda("agenda_v2", nombre=30, calle=30, ciudad=20)
agenda.guardar_contacto(nombre="Ana Perez", calle="San Martin 2301", ciudad="Caseros")
agenda.guardar_contacto(nombre="Juan Pedro Ares", calle="Arenales 1208", ciudad="Martín Coronado")
agenda.guardar_contacto(nombre="Sofia Garcia", calle="", ciudad="San Justo")
agenda.guardar_contacto(nombre="Luis Gonzalez", ciudad="San Justo", calle="Arieta 1234")

In [ ]:
campos = agenda.campos()
for registro in agenda:
    for campo, valor in zip(campos, registro):
        print(f"{campo}: {valor}")
    print()

In [ ]:
agenda.guardar_contacto(nombre="Pedro Sanchez", calle="Ntra Sra de la Merced 1528 piso 10 depto C", ciudad="Caseros")
agenda.guardar_contacto(
    nombre="Leonardo Gonzalo Medrano Fernandez", calle="Cabildo 2411", ciudad="Ciudad Autónoma de Buenos Aires"
)

In [ ]:
campos = agenda.campos()
for registro in agenda:
    for campo, valor in zip(campos, registro):
        print(f"{campo}: {valor}")
    print()

### Otras variantes
- Registros de longitud fija y longitud de campos variables: cada campo va precedido por el tamaño o se usa un separador entre campos, pero la la longitud total del registro sigue siendo fija
    - cantidad de campos fija
    - cantidad de campos variables
- Registros de longitud variables: 
    - cantidad de campos fija: cada campo va precedido por su tamaño
    - cantidad de campos variables: cada registro va precedido por su tamaño y los campos pueden estar precedidos por su tamaño, estar separados por un caracter especial o ser de tamaño fijo

In [ ]:
class Agenda:
    def __init__(self, archivo, longitud_registro, *campos):
        """
        Inicializa la agenda con una longitud fija de registro, una cantidad fija de campos y un archivo binario.
        :param archivo: Nombre del archivo binario donde se guardan los registros.
        :param longitud_registro: Longitud fija de los registros.
        :param campos: Nombres de los campos de la agenda.
        """
        self._len_registro = longitud_registro
        self._cantidad_campos = len(campos)
        self._campos = campos
        self._archivo = archivo
        try:
            # obtengo el tamaño del archivo si es mayor que cero ya teniamos registros guardados
            tam_archivo = os.path.getsize(archivo)
            self._cant_registros = tam_archivo // self._len_registro
        except FileNotFoundError:
            self._cant_registros = 0

    def agregar_registro(self, **datos):
        """
        Agrega un registro a la agenda.
        :param datos: Valores de la forma campo:valor.
        """
        if len(datos) != self._cantidad_campos:
            raise AgendaException(f"Se esperaban {self._cantidad_campos} campos, pero se recibieron {len(datos)}.")
        campos_validos = set(campos)
        campos_recibidos = set(datos.keys())
        if campos_validos != campos_recibidos:
            raise AgendaException("Los campos recibidos no coinciden con los campos definidos en la agenda.")
        registro = ""
        for valor in datos.values():
            valor_str = str(valor)
            longitud_campo = len(valor_str)
            registro += f"{longitud_campo:02d}{valor_str}"  # 02d -> 2 caracteres, relleno con ceros a la izquierda

        if len(registro) > self._len_registro + 2 * self._cantidad_campos:
            raise AgendaException(f"El registro excede la longitud fija permitida: {self.longitud_registro}.")

        registro = registro.ljust(self._len_registro)
        with open(self._archivo, "ab") as archivo:
            archivo.write(registro.encode())

    def __iter__(self):
        return AgendaIterator(self)

In [ ]:
class AgendaIterator:

    def __init__(self, agenda):
        self._agenda = agenda
        self._index = 0

    def __next__(self):
        if self._index >= self._agenda._cant_registros:
            raise StopIteration

        with open(self._agenda._archivo, "rb") as fd:
            posicion = self._index * self._agenda._len_registro
            fd.seek(posicion)
            registro = fd.read(self._agenda._len_registro)
            if len(registro) != self._agenda._len_registro:
                raise StopIteration("Registro corrupto.")
            self._index += 1
            resultado = {}
            for campo in self._agenda._campos:
                longitud_campo = int(registro[:2])
                valor = registro[2 : 2 + longitud_campo]
                resultado[campo] = valor.decode()
                registro = registro[2 + longitud_campo :]

        return resultado

In [ ]:
# Ejemplo de uso
campos = ("Nombre", "Apellido", "Ciudad")
agenda = Agenda("agenda_v3", 80, *campos)
agenda.agregar_registro(Nombre="Juan", Apellido="Perez", Ciudad="Ciudadela")
agenda.agregar_registro(Nombre="Ana", Apellido="Garcia", Ciudad="Liniers")
agenda.agregar_registro(Nombre="Pedro", Apellido="Sanchez", Ciudad="Saenz Pena")

In [ ]:
for registro in agenda:
    for campo, valor in registro.items():
        print(f"{campo}: {valor}")
    print()

with open("agenda_v3", "rb") as archivo:
    for registro in archivo:
        print(registro)

### Acceso a los datos
- **Secuenciales:** los registros se leen desde el principio hasta el final del archivo, de tal forma que para leer un registro se leen todos los que preceden.

- **Directo:** cada registro puede leerse / escribirse de forma directa solo con expresar su dirección en el fichero por él numero relativo del registro o por transformaciones de la clave de registro en él numero relativo del registro a acceder.

- **Por Índice:** se accede indirectamente a los registros por su clave, mediante consulta secuenciales a una tabla que contiene la clave y la dirección relativa de cada registro, y posterior acceso directo al registro.

### Archivos CSV
Longitud de campo variables, los campos se separan por coma (`,`) o por algún otro caracter.
Todos los registros tienen la misma cantidad de campos. En cada línea se guarda un registro y la primera línea puede tener un encabezado con los nombres de los campos

In [ ]:
import csv
import os


class Agenda_csv:
    """Agenda sobre archivos csv, con campos definidos por el usuario"""

    def __init__(self, archivo, *campos):
        self._archivo = archivo
        self._campos = campos

        # Si el archivo no existe escribo en el encabezado
        try:
            os.path.getsize(archivo)
        except FileNotFoundError:
            with open(archivo, "w", newline="") as datos:
                writer = csv.DictWriter(datos, fieldnames=campos, delimiter=";")
                writer.writeheader()

    def guardar_contacto(self, **datos):
        with open(self._archivo, "a", newline="") as archivo:
            writer = csv.DictWriter(archivo, fieldnames=self._campos, delimiter=";")
            writer.writerow(datos)

In [ ]:
agenda = Agenda_csv("agenda.csv", "nombre", "apellido", "direccion", "telefono", "ciudad")
agenda.guardar_contacto(nombre="Ana", apellido="Perez", direccion="Amenabar 1457", telefono="1547892176", ciudad="CABA")
agenda.guardar_contacto(apellido="Videla", nombre="Lucas", telefono="1521264741")
agenda.guardar_contacto(nombre="Martín", apellido="Albarracín", direccion="Urquiza 1324", ciudad="Caseros")

In [ ]:
with open("agenda.csv", "r") as agenda:
    reader = csv.DictReader(agenda, delimiter=";")
    for linea in reader:
        print("Apellido: " + linea["apellido"])
        print("Nombre: " + linea["nombre"])
        print("Telefono: " + linea["telefono"])
        print()

In [ ]:
help(csv.reader)

### Agenda JSON
Recordemos que en JSON solo podemos tener un único diccionario, por lo que para mantener nuestra agenda vamos a seguir la siguiente estrategia. El diccionario va tener como primer par (clave, valor) la cantidad de registros almacenados en la agenda y luego cada clave será el número de registro, iniciando en 0 y el valor todos los datos correspondientes.
Esta estrategia nos permitirá tener registros con campos totalmente diferentes

In [ ]:
import json
import os


class AgendaJson:
    def __init__(self, archivo):
        self._archivo = archivo

        # Si el archivo no existe lo creamos
        try:
            os.path.getsize(archivo)
        except FileNotFoundError:
            agenda = {"cantidad_registros": 0}
            with open(self._archivo, "a") as contenedor:
                json.dump(agenda, contenedor)

    def guardar_contacto(self, **datos):
        with open(self._archivo, "r") as contenedor:
            agenda = json.load(contenedor)
            cantidad = agenda["cantidad_registros"]
        agenda["cantidad_registros"] = cantidad + 1
        agenda[cantidad] = datos
        with open(self._archivo, "w") as contenedor:
            json.dump(agenda, contenedor)

    def __iter__(self):
        """Devuelve un iterador para la agenda"""
        return AgendaJson_Iterator(self)


class AgendaJson_Iterator:
    """Iterador para la agenda"""

    def __init__(self, agenda):
        self._agenda = agenda
        with open(self._agenda._archivo, "r") as contenedor:
            dicc = json.load(contenedor)
        self._cantidad_registros = dicc["cantidad_registros"]
        self._index = 0

    def __next__(self):
        if self._index < self._cantidad_registros:
            with open(self._agenda._archivo, "r") as contenedor:
                dicc = json.load(contenedor)
                registro = dicc[str(self._index)]
                self._index += 1
        else:
            raise StopIteration
        return registro

In [ ]:
agenda = AgendaJson("agenda.json")
agenda.guardar_contacto(
    nombre="Ana",
    segundo_nombre="Laura",
    apellido="Perez",
    direccion="Amenabar 1457",
    telefono="1547892176",
    ciudad="CABA",
    CP="C1426AJY",
)
agenda.guardar_contacto(apellido="Videla", nombre="Lucas", telefono="1521264741", estado_civil="casado")
agenda.guardar_contacto(nombre="Martín", apellido="Albarracín", direccion="Urquiza 1324", ciudad="Caseros")

In [ ]:
for data in agenda:
    for clave, valor in data.items():
        print("{} = {}".format(clave, valor))
    print()

In [ ]:
###### Limpiar directorio. Ejecutar con cuidado
import os

archivos = os.listdir()
conservar = [".ipynb_checkpoints", "Organización de Datos.ipynb", "rise.css"]

for arch in archivos:
    if arch not in conservar:
        os.remove(arch)